# Python 101
## XII. Command line interfaces

---

## I. `sys.argv` 

In [1]:
import sys

['C:\\Users\\fulibacsi\\Anaconda3\\lib\\site-packages\\ipykernel_launcher.py',
 '-f',
 'C:\\Users\\fulibacsi\\AppData\\Roaming\\jupyter\\runtime\\kernel-520c33cd-d205-427d-bcc6-4151d3fbeea6.json']

---

## II. `argparse`

In [4]:
import argparse

---

## III. `click`

In [ ]:
import click

---

## IV. Exercises

### 1.

### 2.